# Deep Learning Grid Search

In this project, we will learn how to use the scikit-learn grid search capability.

We are going to learn the following topics:

* How to use Keras models in scikit-learn.
* How to use grid search in scikit-learn.
* How to tune batch size and training epochs.
* How to tune learning rate
* How to tune network weight initialization.
* How to tune activation functions.
* How to tune dropout regularization.
* How to tune the number of neurons in the hidden layer.

In [1]:
import sys
import pandas
import numpy
import sklearn
import keras

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Using Theano backend.
C:\Programdata\anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Python: 2.7.13 |Continuum Analytics, Inc.| (default, May 11 2017, 13:17:26) [MSC v.1500 64 bit (AMD64)]
Pandas: 0.21.0
Numpy: 1.14.0
Sklearn: 0.19.1
Keras: 2.0.5


In [2]:
import pandas as pd
import numpy as np

# import the uci pima indians diabetes dataset
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(url, names = names)

In [3]:
# Describe the dataset
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df[df['glucose_concentration'] == 0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [5]:
# Preprocess the data, mark zero values as NaN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
# Drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in df
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset.shape)

(392L, 9L)


In [10]:
# split into input (X) and an output (Y)
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [12]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392L, 8L)
(392L,)
[0 1 1 1 1]


In [13]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [14]:
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [15]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-9.063045e-18,1.132881e-17,-4.531523e-16,1.087565e-16,1.064908e-16,1.631348e-16,1.812609e-17,1.110223e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [16]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [17]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] epochs=10, batch_size=10 ........................................
Epoch 1/10
261/261 [==============================] - 0s - loss: 0.6333 - acc: 0.6897     
Epoch 2/10
261/261 [==============================] - 0s - loss: 0.4842 - acc: 0.6973     
Epoch 3/10
261/261 [==============================] - 0s - loss: 0.4422 - acc: 0.6973     
Epoch 4/10
261/261 [==============================] - 0s - loss: 0.4247 - acc: 0.6973     
Epoch 5/10
261/261 [==============================] - 0s - loss: 0.4139 - acc: 0.8084     
Epoch 6/10
261/261 [==============================] - 0s - loss: 0.4024 - acc: 0.8238     
Epoch 7/10
261/261 [==============================] - 0s - loss: 0.3962 - acc: 0.8391     
Epoch 8/10
261/261 [==============================] - 0s - loss: 0.3886 - acc: 0.8276     
Epoch 9/10
261/261 [==============================] - 0s - loss: 0.3891 - acc: 0.8161     
Epoch 10/10
 10/261 [>............................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 0s - loss: 0.6607 - acc: 0.7280     
Epoch 2/10
261/261 [==============================] - 0s - loss: 0.6058 - acc: 0.7816     
Epoch 3/10
261/261 [==============================] - 0s - loss: 0.5757 - acc: 0.7854     
Epoch 4/10
261/261 [==============================] - 0s - loss: 0.5311 - acc: 0.8123     
Epoch 5/10
261/261 [==============================] - 0s - loss: 0.5161 - acc: 0.8008     
Epoch 6/10
261/261 [==============================] - 0s - loss: 0.5176 - acc: 0.7969     
Epoch 7/10
261/261 [==============================] - 0s - loss: 0.5040 - acc: 0.7969     
Epoch 8/10
261/261 [==============================] - 0s - loss: 0.4814 - acc: 0.7931     
Epoch 9/10
261/261 [==============================] - 0s - loss: 0.4738 - acc: 0.8008     
Epoch 10/10
 10/261 [>.............................] - ETA: 0s[CV] ... epochs=10, batch_size=10, score=0.770992366412, total=   4.6s
[CV] epochs=10, batch_size=10 ..................

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.3s remaining:    0.0s


Epoch 1/10
262/262 [==============================] - 0s - loss: 0.6352 - acc: 0.6450     
Epoch 2/10
262/262 [==============================] - 0s - loss: 0.5273 - acc: 0.6527     
Epoch 3/10
262/262 [==============================] - 0s - loss: 0.5015 - acc: 0.6870     
Epoch 4/10
262/262 [==============================] - 0s - loss: 0.4985 - acc: 0.7519     
Epoch 5/10
262/262 [==============================] - 0s - loss: 0.4887 - acc: 0.7748     
Epoch 6/10
262/262 [==============================] - 0s - loss: 0.4835 - acc: 0.7710     
Epoch 7/10
262/262 [==============================] - 0s - loss: 0.4791 - acc: 0.7824     
Epoch 8/10
262/262 [==============================] - 0s - loss: 0.4739 - acc: 0.7901     
Epoch 9/10
262/262 [==============================] - 0s - loss: 0.4707 - acc: 0.7901     
Epoch 10/10
 10/262 [>.............................] - ETA: 0s[CV] ... epochs=10, batch_size=10, score=0.838461532043, total=   3.8s
[CV] epochs=50, batch_size=10 ..................

...

262/262 [==============================] - 0s - loss: 0.5499 - acc: 0.6527     
Epoch 5/10
262/262 [==============================] - 0s - loss: 0.5239 - acc: 0.6527     
Epoch 6/10
262/262 [==============================] - 0s - loss: 0.5155 - acc: 0.6527     
Epoch 7/10
262/262 [==============================] - 0s - loss: 0.5078 - acc: 0.6527     
Epoch 8/10
262/262 [==============================] - 0s - loss: 0.5025 - acc: 0.6527     
Epoch 9/10
262/262 [==============================] - 0s - loss: 0.4976 - acc: 0.6527     
Epoch 10/10
 40/262 [===>..........................] - ETA: 0s[CV] ... epochs=10, batch_size=40, score=0.699999988079, total=   2.9s
[CV] epochs=50, batch_size=40 ........................................
Epoch 1/50
261/261 [==============================] - 0s - loss: 0.6688 - acc: 0.6513     
Epoch 2/50
261/261 [==============================] - 0s - loss: 0.5790 - acc: 0.7241     
Epoch 3/50
261/261 [==============================] - 0s - loss: 0.4898 - acc: 

261/261 [==============================] - 0s - loss: 0.3627 - acc: 0.8352     
Epoch 13/100
261/261 [==============================] - 0s - loss: 0.3584 - acc: 0.8352     
Epoch 14/100
261/261 [==============================] - 0s - loss: 0.3537 - acc: 0.8391     
Epoch 15/100
261/261 [==============================] - 0s - loss: 0.3486 - acc: 0.8391     
Epoch 16/100
261/261 [==============================] - 0s - loss: 0.3455 - acc: 0.8391     
Epoch 17/100
261/261 [==============================] - 0s - loss: 0.3434 - acc: 0.8467     
Epoch 18/100
261/261 [==============================] - 0s - loss: 0.3401 - acc: 0.8429     
Epoch 19/100
261/261 [==============================] - 0s - loss: 0.3385 - acc: 0.8391     
Epoch 20/100
261/261 [==============================] - 0s - loss: 0.3344 - acc: 0.8391     
Epoch 21/100
261/261 [==============================] - 0s - loss: 0.3319 - acc: 0.8391     
Epoch 22/100
261/261 [==============================] - 0s - loss: 0.3300 - acc: 0.

261/261 [==============================] - 0s - loss: 0.2535 - acc: 0.8851     
Epoch 84/100
261/261 [==============================] - 0s - loss: 0.2481 - acc: 0.8889     
Epoch 85/100
261/261 [==============================] - 0s - loss: 0.2509 - acc: 0.8736     
Epoch 86/100
261/261 [==============================] - 0s - loss: 0.2534 - acc: 0.8774     
Epoch 87/100
261/261 [==============================] - 0s - loss: 0.2469 - acc: 0.8851     
Epoch 88/100
261/261 [==============================] - 0s - loss: 0.2480 - acc: 0.8927     
Epoch 89/100
261/261 [==============================] - 0s - loss: 0.2405 - acc: 0.8812     
Epoch 90/100
261/261 [==============================] - 0s - loss: 0.2546 - acc: 0.8966     
Epoch 91/100
261/261 [==============================] - 0s - loss: 0.2430 - acc: 0.8889     
Epoch 92/100
261/261 [==============================] - 0s - loss: 0.2442 - acc: 0.8927     
Epoch 93/100
261/261 [==============================] - 0s - loss: 0.2348 - acc: 0.

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.7min finished


Epoch 1/100
392/392 [==============================] - 0s - loss: 0.6723 - acc: 0.6480     
Epoch 2/100
392/392 [==============================] - 0s - loss: 0.5787 - acc: 0.6684     
Epoch 3/100
392/392 [==============================] - 0s - loss: 0.5089 - acc: 0.6684     
Epoch 4/100
392/392 [==============================] - 0s - loss: 0.4887 - acc: 0.6684     
Epoch 5/100
392/392 [==============================] - 0s - loss: 0.4738 - acc: 0.6684     
Epoch 6/100
392/392 [==============================] - 0s - loss: 0.4648 - acc: 0.7628     
Epoch 7/100
392/392 [==============================] - 0s - loss: 0.4607 - acc: 0.7908     
Epoch 8/100
392/392 [==============================] - 0s - loss: 0.4533 - acc: 0.7934     
Epoch 9/100
392/392 [==============================] - 0s - loss: 0.4445 - acc: 0.8061     
Epoch 10/100
392/392 [==============================] - 0s - loss: 0.4356 - acc: 0.8036     - ETA: 0s - loss: 0.4487 - acc: 0.780
Epoch 11/100
392/392 [====================

In [23]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] learn_rate=0.001, dropout_rate=0.0 ..............................
[CV]  learn_rate=0.001, dropout_rate=0.0, score=0.740458024367, total=   7.4s
[CV] learn_rate=0.001, dropout_rate=0.0 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.4s remaining:    0.0s


[CV]  learn_rate=0.001, dropout_rate=0.0, score=0.778625960113, total=   5.4s
[CV] learn_rate=0.001, dropout_rate=0.0 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.9s remaining:    0.0s


[CV]  learn_rate=0.001, dropout_rate=0.0, score=0.830769236271, total=   4.8s
[CV] learn_rate=0.01, dropout_rate=0.0 ...............................


...

[CV]  learn_rate=0.1, dropout_rate=0.0, score=0.815384612634, total=   4.4s
[CV] learn_rate=0.001, dropout_rate=0.1 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   45.0s remaining:    0.0s


[CV]  learn_rate=0.001, dropout_rate=0.1, score=0.732824443861, total=   5.7s
[CV] learn_rate=0.001, dropout_rate=0.1 ..............................
[CV]  learn_rate=0.001, dropout_rate=0.1, score=0.770992370507, total=   7.3s
[CV] learn_rate=0.001, dropout_rate=0.1 ..............................
[CV]  learn_rate=0.001, dropout_rate=0.1, score=0.838461536628, total=   6.1s
[CV] learn_rate=0.01, dropout_rate=0.1 ...............................
[CV]  learn_rate=0.01, dropout_rate=0.1, score=0.717557246448, total=   5.9s
[CV] learn_rate=0.01, dropout_rate=0.1 ...............................
[CV]  learn_rate=0.01, dropout_rate=0.1, score=0.732824434761, total=   5.5s
[CV] learn_rate=0.01, dropout_rate=0.1 ...............................
[CV]  learn_rate=0.01, dropout_rate=0.1, score=0.846153841569, total=   6.8s
[CV] learn_rate=0.1, dropout_rate=0.1 ................................
[CV]  learn_rate=0.1, dropout_rate=0.1, score=0.694656491507, total=   4.8s
[CV] learn_rate=0.1, dropout_rate

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.5min finished


Best: 0.783163272148, using {'learn_rate': 0.001, 'dropout_rate': 0.0}
0.783163272148 (0.0494308487858) with: {'learn_rate': 0.001, 'dropout_rate': 0.0}
0.767857150004 (0.0432823764463) with: {'learn_rate': 0.01, 'dropout_rate': 0.0}
0.757653056815 (0.0573177537304) with: {'learn_rate': 0.1, 'dropout_rate': 0.0}
0.780612251132 (0.0300745493047) with: {'learn_rate': 0.001, 'dropout_rate': 0.1}
0.765306121537 (0.0337785679152) with: {'learn_rate': 0.01, 'dropout_rate': 0.1}
0.742346945086 (0.0317644706476) with: {'learn_rate': 0.1, 'dropout_rate': 0.1}
0.767857143009 (0.0301926398124) with: {'learn_rate': 0.001, 'dropout_rate': 0.2}
0.775510210316 (0.0531929367344) with: {'learn_rate': 0.01, 'dropout_rate': 0.2}
0.724489800176 (0.0493287378951) with: {'learn_rate': 0.1, 'dropout_rate': 0.2}


In [25]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV]  activation=softmax, init=uniform, score=0.740458024367, total=  14.1s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.2s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.755725200395, total=   4.1s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.4s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.815384621804, total=   4.4s
[CV] activation=softmax, init=normal .................................


...

[CV]  activation=softmax, init=zero, score=0.699999988079, total=   4.1s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   53.0s remaining:    0.0s


[CV]  activation=relu, init=uniform, score=0.725190854255, total=   4.6s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.770992369597, total=   4.1s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.85384615568, total=   4.4s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.732824434761, total=   4.1s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.770992369597, total=   4.5s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.807692307692, total=   4.5s
[CV] activation=relu, init=zero ......................................
[CV] . activation=relu, init=zero, score=0.610687024266, total=   3.9s
[CV] activation=relu, init=zero .....................................

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  2.8min finished


Best: 0.793367353173, using {'activation': 'linear', 'init': 'uniform'}
0.770408171628 (0.0494308487858) with: {'activation': 'softmax', 'init': 'uniform'}
0.7295918416 (0.0432823764463) with: {'activation': 'softmax', 'init': 'normal'}
0.668367345874 (0.0573177537304) with: {'activation': 'softmax', 'init': 'zero'}
0.783163271844 (0.0300745493047) with: {'activation': 'relu', 'init': 'uniform'}
0.770408166763 (0.0337785679152) with: {'activation': 'relu', 'init': 'normal'}
0.668367345874 (0.0317644706476) with: {'activation': 'relu', 'init': 'zero'}
0.785714293165 (0.0301926398124) with: {'activation': 'tanh', 'init': 'uniform'}
0.790816332157 (0.0531929367344) with: {'activation': 'tanh', 'init': 'normal'}
0.668367345874 (0.0493287378951) with: {'activation': 'tanh', 'init': 'zero'}


In [26]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................
[CV] ....... neuron1=4, neuron2=2, score=0.770992364592, total=   7.3s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.3s remaining:    0.0s


[CV] ....... neuron1=4, neuron2=2, score=0.763358790001, total=   5.6s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.0s remaining:    0.0s


[CV] ....... neuron1=4, neuron2=2, score=0.830769236271, total=   5.4s
[CV] neuron1=4, neuron2=4 ............................................


...

[CV] ....... neuron1=4, neuron2=8, score=0.830769236271, total=   4.3s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   47.7s remaining:    0.0s


[CV] ....... neuron1=8, neuron2=2, score=0.763358784086, total=   4.2s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ....... neuron1=8, neuron2=2, score=0.763358790001, total=   3.1s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ....... neuron1=8, neuron2=2, score=0.838461545797, total=   3.5s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ....... neuron1=8, neuron2=4, score=0.763358774986, total=   3.6s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ....... neuron1=8, neuron2=4, score=0.763358790001, total=   3.4s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ....... neuron1=8, neuron2=4, score=0.830769236271, total=   3.1s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ....... neuron1=8, neuron2=8, score=0.763358784086, total=   4.1s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.0min finished


Best: 0.790816335198, using {'neuron1': 16, 'neuron2': 2}
0.78826531114 (0.0494308487858) with: {'neuron1': 4, 'neuron2': 2}
0.785714291644 (0.0432823764463) with: {'neuron1': 4, 'neuron2': 4}
0.785714290124 (0.0573177537304) with: {'neuron1': 4, 'neuron2': 8}
0.788265314181 (0.0300745493047) with: {'neuron1': 8, 'neuron2': 2}
0.785714290124 (0.0337785679152) with: {'neuron1': 8, 'neuron2': 4}
0.785714293165 (0.0317644706476) with: {'neuron1': 8, 'neuron2': 8}
0.790816335198 (0.0301926398124) with: {'neuron1': 16, 'neuron2': 2}
0.790816330636 (0.0531929367344) with: {'neuron1': 16, 'neuron2': 4}
0.785714293165 (0.0493287378951) with: {'neuron1': 16, 'neuron2': 8}


In [27]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [28]:
print(y_pred.shape)

(392L, 1L)


In [29]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [30]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7806122448979592
             precision    recall  f1-score   support

          0       0.81      0.89      0.84       262
          1       0.71      0.57      0.63       130

avg / total       0.77      0.78      0.77       392

